# 事前準備

*   タイムアウト切れ回避対応
*   共通パスの指定
*   GDrive接続
*   BigQuery接続





In [5]:
#タイムアウト切れ回避対応
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

実行設定値

In [6]:
#クライアントID, 実行日
clientId = "aoscs"
execute_date= "20230517"

Gドライブフォルダパス

In [7]:
#デフォルトGドライブパスを指定
fldContentDrive = '/content/drive/'
fldContentDriveMyDrive = fldContentDrive + 'MyDrive/'

#Colab Notebooks
fldContentDriveMyDriveColabNotebooks = fldContentDriveMyDrive + 'ColabNotebooks/'
fldContentDriveMyDriveColabNotebooksData = fldContentDriveMyDriveColabNotebooks + 'data/'
fldContentDriveMyDriveColabNotebooksDataInput = fldContentDriveMyDriveColabNotebooksData + 'input/' + clientId + "_" + execute_date + '/' 
fldContentDriveMyDriveColabNotebooksDataOutput = fldContentDriveMyDriveColabNotebooksData + 'output/' + clientId + "_" + execute_date + '/'

print(fldContentDriveMyDriveColabNotebooksDataInput)
print(fldContentDriveMyDriveColabNotebooksDataOutput)

/content/drive/MyDrive/ColabNotebooks/data/input/aoscs_20230517/
/content/drive/MyDrive/ColabNotebooks/data/output/aoscs_20230517/


Gドライブ接続

In [8]:
from google.colab import drive
drive.mount(fldContentDrive)

Mounted at /content/drive/


インプット、アウトプットフォルダを作成

In [11]:
import os

#インプットプットフォルダを作成する
if not os.path.exists(fldContentDriveMyDriveColabNotebooksDataInput):
 os.mkdir(fldContentDriveMyDriveColabNotebooksDataInput)

#アウトプットフォルダを作成する
if not os.path.exists(fldContentDriveMyDriveColabNotebooksDataOutput):
  os.mkdir(fldContentDriveMyDriveColabNotebooksDataOutput)

BigQueryへ連携

In [12]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'bwing-230309'
location = 'asia-northeast1'

client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

 トークナイズする説明文章をBqから取得してCSV化

In [13]:
sql = "SELECT * FROM `bwing-230309.attachment_" + clientId + ".v_parse_description_4_tokenized`"
qr = client.query(sql)
df = qr.to_dataframe()
df.to_csv(fldContentDriveMyDriveColabNotebooksDataInput + "bq_products_desc_" + clientId + "_" + execute_date + ".csv", index = False)

インプットファイル名を確認

In [14]:
#Gドライブの インプットフォルダからファイル名を調査
import glob
files = glob.glob(fldContentDriveMyDriveColabNotebooksDataInput + "*")
for file in files:
    print(file)

/content/drive/MyDrive/ColabNotebooks/data/input/aoscs_20230517/bq_products_desc_aoscs_20230517.csv


In [15]:
#インプットファイル名（配列）
file_names = [
    "bq_products_desc_" + clientId + "_" + execute_date + ".csv"
    ]

# 入力するソースを準備する


*   **行単位のフォーマット(CSV)**： 
    <br>---------------------------------------------------------
    <br>{行番号} , {固有ID} , {名称} , {解析する文章}   (カンマ区切り）
    <br>行番号：　通し番号でよい
    <br>固有ID：　商品IDなど
    <br>名称:　商品名など→固有でなくてよい
    <br>解析する文章:　商品説明の要約文など
    <br>---------------------------------------------------------


*   **※クライアントなどから連携されたデータをCSV化して、inputフォルダへ格納しておく**


In [16]:
#ファイルの有無をチェック
%ls {fldContentDriveMyDriveColabNotebooksDataInput}

bq_products_desc_aoscs_20230517.csv


# ストップワード登録


*   **行単位の単語**： 
    <br>---------------------------------------------------------
    <br>{単語}
    <br>---------------------------------------------------------


*   **※先に調査し、不要なワードを格納しておく**
*   ストップワードファイルの読み込みを配列化する


In [19]:
input_fld = fldContentDriveMyDriveColabNotebooksData + 'input/'

stop_words_file_name = "stop_words"
input_file_path = input_fld + clientId + "_stop_words/" + stop_words_file_name
stop_words_file = []

with open(input_file_path) as f:
  for index, line in enumerate(f):
    stop_words_file.append(line.replace('\n', ''))

print(stop_words_file)

['使用', '詳細', '商品', '商品詳細']


# 形態素解析ライブラリを準備

GiNZAとspaCyをインストールして、
正常に動作するかを確認

In [20]:
!pip install ginza==4.0.5 ja-ginza 
!pip install sortedcontainers~=2.1.0
import pkg_resources, imp
imp.reload(pkg_resources)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 9.1 MB/s eta 0:00:00
  Created wheel for ginza: filename=ginza-4.0.5-py3-none-any.whl size=15878 sha256=9032c1c0f417a53129befc2270a525f3a8d3a25bd3618f5c89771f04067a960e
  Stored in directory: /root/.cache/pip/wheels/b0/fb/e4/fe3e5425cf74396161f545392e48a6c78a7f221ec44dc6dc19
  Created wheel for ja-ginza: filename=ja_gin

<module 'pkg_resources' from '/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py'>

In [21]:
import spacy
import ginza
nlp = spacy.load("ja_ginza")
ginza.set_split_mode(nlp, "C")
doc = nlp("彼女はペンパイナッポーアッポーペンと恋ダンスを踊った。")

for tok in doc:
    print(
        tok.i,  # 親ドキュメント内のトークンのインデックス。
        tok.text,  # 逐語的なテキストコンテンツ

        # 逐語的なテキストコンテンツ（Token.textと同じ）
        # 主に他の属性との一貫性のために存在します。
        #tok.orth_,
        #tok.lemma_,  # 語尾変化のない接尾辞のない、トークンの基本形式
        
        tok.pos_,  # 品詞（英語の大文字）
        tok.tag_,  # きめの細かい品詞
        tok.head.i, # headはこのトークンの構文上の親、または「ガバナー」
        tok.dep_,  # 構文従属関係

        # トークンの基準、つまりトークンテキストの正規化された形式
        # 通常、言語のトークナイザー例外またはノルム例外で設定されます。
        tok.norm_,
        
        ## 名前付きエンティティタグのIOBコード
        ## 「B」はトークンがエンティティを開始することを意味し、「I」はそれがエンティティの内部にあることを意味し、
        ## 「O」はそれがエンティティの外部にあることを意味し、「」はエンティティタグが設定されていないことを意味します。
        #tok.ent_iob_,
        #tok.ent_type_,  # 名前付きエンティティタイプ
    )

0 彼女 PRON 代名詞 7 nsubj 彼女
1 は ADP 助詞-係助詞 0 case は
2 ペンパイナッポーアッポーペン NOUN 名詞-普通名詞-一般 7 advmod ペンパイナッポーアッポーペン
3 と ADP 助詞-格助詞 2 case と
4 恋 NOUN 名詞-普通名詞-サ変可能 5 compound 恋
5 ダンス NOUN 名詞-普通名詞-サ変可能 7 obj ダンス
6 を ADP 助詞-格助詞 5 case を
7 踊っ VERB 動詞-一般 7 ROOT 踊っ
8 た AUX 助動詞 7 aux た
9 。 PUNCT 補助記号-句点 7 punct .


# 入力ファイルに対して形態素解析を行い、ファイルに結果を書き込む

In [22]:
#文字列の正規化の定義
#表記揺れを減らします。今回はneologdの正規化処理を一部改変したものを利用します。 処理の詳細はリンク先を参照してください。
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

In [23]:
def shape_word_by_type(string, type):

    #名詞-普通名詞-一般
    stop_words_name_normal_ge = ['シャツ']

    #名詞-普通名詞-サ変可能
    stop_words_name_normal_sa = ['使用']

    #名詞-固有名詞-人名-一般、名詞-固有名詞-地名-一般
    stop_words_uniq_human = ['']

    #名詞-普通名詞-形状詞可能
    stop_words_name_form = ['詳細']

    #形状詞-一般
    stop_words_form_ge = ['']

    #副詞
    stop_words_sub = ['']

    #ここでストップワードを登録
    stop_words_add = ['']

    #ストップワードを結合
    stop_words_all = stop_words_name_normal_sa + stop_words_name_normal_ge + stop_words_uniq_human + stop_words_name_form + stop_words_form_ge + stop_words_sub + stop_words_file + stop_words_add

    #ストップワードから不要なワードを削除
    stop_words_all.remove('')

    if string in stop_words_all:
      return False
    
    #分類判定
    if type in [
        '名詞-普通名詞-一般', 
        '名詞-普通名詞-サ変可能', 
        '名詞-固有名詞-人名-一般', 
        '名詞-固有名詞-地名-一般', 
        '名詞-普通名詞-形状詞可能', 
        '形状詞-一般', 
        '副詞'
      ]:
      return True
    else:
      return False

def remove_brackets(text):
    text = re.sub(r"(^【[^】]*】)|(【[^】]*】$)", "", text)
    return text

def normalize_text(text):
    #assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

def read_id_sentence_line(line):
    line = normalize_text(remove_brackets(line))
    row = line.split(',')
    n = row[0]
    id = row[1]
    name = row[2]
    sentence = row[3]
    return n, id, name, sentence

def morphological_analize(input_file_path, output_file_path, dt):

  with open(input_file_path) as f:
    toks = []
    for index, line in enumerate(f):

      print("【", index, "】ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー")

      #CSVを分解
      n, id, name, sentence = read_id_sentence_line(line)
      
      #Sample tasting
      #sentence = "暑さをクールダウンさせる加工素材を使用した、ビズシーズンにおすすめのスタンダードワイシャツ"
      
      #形態素解析
      doc = nlp(sentence)
      
      #解析結果箱を初期化
      arr_tok_text = []
      arr_tok_tag_ = []
      str_tok_texts = ''
      str_tok_tags = ''

      #形態素解析結果を配列化
      for tok in doc:
          #print(
          #    tok.text,  # 逐語的なテキストコンテンツ
          #    tok.tag_,  # きめの細かい品詞)
          if shape_word_by_type(tok.text, tok.tag_):
            arr_tok_text.append(tok.text)
            arr_tok_tag_.append(tok.tag_)
          
      #解析結果配列をカンマでつないで文字列化
      str_tok_texts = ','.join(arr_tok_text)
      str_tok_tags = ','.join(arr_tok_tag_)

      #０行目はヘッダー
      if index == 0:
        mr = 'cdt|client_id|item_id|name|tokenized|classification|description_summary'

      #入力値と結果値をそれぞれパイプでつないで文字列化
      if index != 0:
        mr = dt + '|' + clientId + '|' + id + '|' + name + '|' + str_tok_texts.replace('\n', '') + '|' + str_tok_tags.replace('\n', '') + '|' + sentence

      #格納する
      toks.append(mr)
      print(mr)

  with open(output_file_path, mode='w') as f:
    for index, s in enumerate(toks):
      #print(s)
      f.write(s+'\n')


## 解析を実行

*   Timestampを設定
*   入力ファイル名と出力先指定を
*   inference_summaryを実行(ファイルごとにループ上でCall)

In [24]:
#Timestamp
from datetime import datetime
dt = datetime.now() # Getting the current date and time
ts = datetime.timestamp(dt) # getting the timestamp
print("Date and time is:", dt)
#timestamp to string
sｄｔ = dt.strftime("%Y-%m-%d %H:%M:%S")
sdt = sdt.replace(' ', '_')

#形態素解析
input_fld = fldContentDriveMyDriveColabNotebooksDataInput
output_fld = fldContentDriveMyDriveColabNotebooksDataOutput

for f in file_names:
  input_file_path = input_fld + f
  output_file_path = output_fld + "tok_" + f
  morphological_analize(input_file_path, output_file_path, sdt)

ストリーミング出力は最後の 5000 行に切り捨てられました。
【 2822 】ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
2023-05-25_03:52:55|aoscs|66280|スタイリッシュジャケット【tiotio premium】【active expand】|ウォッシャブル,way,ストレッチ,シワ,速乾,tiotio,ウイルス,抗菌,消臭,静電気,プロテクト|名詞-普通名詞-一般,名詞-普通名詞-一般,名詞-普通名詞-一般,名詞-普通名詞-一般,名詞-普通名詞-一般,名詞-普通名詞-一般,名詞-普通名詞-一般,名詞-普通名詞-一般,名詞-普通名詞-サ変可能,名詞-普通名詞-一般,名詞-普通名詞-サ変可能|ウォッシャブル2wayストレッチ防シワ速乾tiotio(抗ウイルス抗菌・消臭静電気プロテクト)
【 2823 】ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
2023-05-25_03:52:55|aoscs|66297|スタイリッシュスラックス【ノータック】【裾上げ済み】|ウォッシャブル,way,ストレッチ|名詞-普通名詞-一般,名詞-普通名詞-一般,名詞-普通名詞-一般|ウォッシャブル2wayストレッチ
【 2824 】ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
2023-05-25_03:52:55|aoscs|66298|スタイリッシュスラックス【ノータック】【裾上げ済み】|ウォッシャブル,way,ストレッチ|名詞-普通名詞-一般,名詞-普通名詞-一般,名詞-普通名詞-一般|ウォッシャブル2wayストレッチ
【 2825 】ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
2023-05-25_03:52:55|aoscs|66299|スタイリッシュスラックス【ノータック】【裾上げ済み】|ウォッシャブル,way,ストレッチ|名詞-普通名詞-一般,名詞-普通名詞-一般,名詞-普通名詞-一般|ウォッシャブル2wayストレッチ
【 2826 】ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
2023-05-25_03:52:55|aoscs|66232|スタイリッシュパンツ【ウォッシャブル】【セットアップ着用可】|ウォッシャブル,way,ストレッチ,ウエスト,ストレッチ

**出力したCSVをDataframe化し、BigQueryへ連携**

In [25]:
from pandas.core.indexing import IndexingError
import csv, os
import pandas as pd
import numpy as np

table = 'attachment_' + clientId + '.import_colab2bq_tokenized_description'
output_fld = fldContentDriveMyDriveColabNotebooksDataOutput

df = pd.DataFrame()
i=0
for f in file_names:
  i += 1
  file_path = output_fld + "tok_" + f
  print("file name:" + file_path)

  #CSVファイルからdfへ変換
  #df = df.append(pd.read_csv(file_path, sep='|'))
  df = pd.read_csv(file_path, sep='|')

  #読み込んだデータの表示
  df.head()

  #bigqueryのテーブルを作成
  if i==1:
    df.to_gbq(table, project_id=project, if_exists="replace")  
  else:
    df.to_gbq(table, project_id=project, if_exists="append")


file name:/content/drive/MyDrive/ColabNotebooks/data/output/aoscs_20230517/tok_bq_products_desc_aoscs_20230517.csv


100%|██████████| 1/1 [00:00<00:00, 1294.54it/s]
